### Libraries:

In [6]:
import pandas as pd

from preprocessing import past_events, prepare_delays, distances_between_hospitals, rolling_sums_and_means, enrich_data, situation_at_all_hospitals
from acquisition import get_weather_data, get_bank_holiday_data

### Reading in the fake data for Assignments, Incidents and Patient Transfers System (PTS)

In [132]:
df_arrived = pd.read_parquet('path/to/your/file/arrived.parquet')
df_incidents = pd.read_parquet('path/to/your/file/incidents.parquet')
df_pts = pd.read_parquet('path/to/your/file/pts.parquet')

### Preparing a dataframe for past events

The past events function takes the Assignments, Incidents and PTS dataframes and merges them together to give an overall representation of the hospital at each timestamp. We output a dataframe that describes the following: 
- hospital
- time
- number of ambulances arriving
- number of specific skills missing from ambulance arrivals (e.g. missing_skill_A, missing_skill_B)
- idea of flow in and out of the hospital using admissions, discharges and transfers (PTS)
- the mean handover delays
- the number of each age band arriving in ambulances
- the number of patients arriving with each responding priority
- the number of patients being transported by the ambulances
- the number of ambulances leaving the queue

For example, at a specifc time there will be a 1 in missing_skill_A if the no crew members on that ambulance have that skill.

In [17]:
df_past_events = past_events(df_arrived, df_incidents, df_pts)
df_past_events.head()

,hospital,time,num_ambulance_arrivals,missing_skill_B,missing_skill_A,missing_skill_E,missing_skill_C,missing_skill_D,flow,past_delay_mins,...,priority_4,priority_5,priority_6,priority_7,priority_8,priority_9,1_patient_transported,2_patients_transported,3_patients_transported,num_ambulance_depatures
0,hospital_A,1953-01-12 18:36:32,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hospital_A,1953-01-12 19:34:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,hospital_A,1953-01-12 20:53:50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,hospital_A,1953-01-12 21:13:45,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,hospital_A,1953-01-12 21:41:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [19]:
# Preparing delays dataframe and outputting the first five rows
delays = prepare_delays(df_arrived)
delays.head()

,time_plus_3,hospital,handover_delay_mins,time_plus_10,time_plus_24
0,1954-01-18 10:16:31,hospital_D,27119.91,1954-01-18 10:16:31,1954-01-18 10:16:31
1,1953-05-27 04:55:49,hospital_D,77760.45,1953-05-27 04:55:49,1953-05-27 04:55:49
2,1953-09-07 08:46:15,hospital_D,0.00,1953-09-07 08:46:15,1953-09-07 08:46:15
3,1953-10-22 09:55:39,hospital_C,168294.37,1953-10-22 09:55:39,1953-10-22 09:55:39
4,1953-10-14 19:29:40,hospital_E,277823.96,1953-10-14 19:29:40,1953-10-14 19:29:40


### Preparing distances dataframe

Using a function from preprocessing.py which calculates the Euclidean distance between each of the 11 hospitals to be used to see whether the number of ambulances arriving at hospitals nearby has an impact on the situation at the current hospital.

In [20]:
distances = distances_between_hospitals(df_arrived)
distances.head()

,hospital,hospital_other,distance
0,hospital_D,hospital_D,0.000000
1,hospital_D,hospital_C,530975.249673
2,hospital_D,hospital_E,123474.217426
3,hospital_D,hospital_B,875690.146913
4,hospital_D,hospital_A,251123.018533


### Preparing three dataframes representing the situation at all hospitals. 

The three dataframes being constructed are to be used as training data; one for the prediction three hours ahead, one for the prediction ten hours ahead and one for the prediction twenty four hours ahead. These dataframes are constructed using the situation at all hospitals function from preprocessing.py. This function combines rolling aggregations of the past events dataframe aligned to the timestamps of the handover delay.

In [21]:
df_for_three_hour_pred = situation_at_all_hospitals(delays, distances, df_past_events, 3)
df_for_ten_hour_pred = situation_at_all_hospitals(delays, distances, df_past_events, 10)
df_for_twenty_four_hour_pred = situation_at_all_hospitals(delays, distances, df_past_events, 24)

### Enriching the situational data with weather data and bank holiday data

Adding on the following information:
- maximum temperature
- minimum temperature
- rainfall
- whether there is a bank holiday or not on that date (Yes/No)

In [22]:
df_for_three_hour_pred = enrich_data(df_for_three_hour_pred, 1950)
df_for_ten_hour_pred = enrich_data(df_for_ten_hour_pred, 1950)
df_for_twenty_four_hour_pred = enrich_data(df_for_twenty_four_hour_pred, 1950)

In [24]:
# Example output of the dataframe for the three hour prediction
df_for_three_hour_pred.head()

,delay_id,time_plus_3,hospital,handover_delay_mins,time_plus_10,time_plus_24,missing_skill_A,missing_skill_B,missing_skill_C,missing_skill_D,...,time,date,day_of_week,month,year,hour_of_day,bank_holiday,tmax,tmin,rain
0,8701.0,1953-01-13 09:17:32,hospital_A,487281.91,1953-01-13 09:17:32,1953-01-13 09:17:32,NaN,NaN,NaN,NaN,...,1953-01-13 09:17:32,1953-01-13,Tuesday,1,1953,9,No,5.4,0.8,21.8
1,8993.0,1953-01-13 12:53:50,hospital_A,0.00,1953-01-13 12:53:50,1953-01-13 12:53:50,NaN,NaN,NaN,NaN,...,1953-01-13 12:53:50,1953-01-13,Tuesday,1,1953,12,No,5.4,0.8,21.8
2,8025.0,1953-01-13 12:56:02,hospital_A,0.00,1953-01-13 12:56:02,1953-01-13 12:56:02,NaN,NaN,NaN,NaN,...,1953-01-13 12:56:02,1953-01-13,Tuesday,1,1953,12,No,5.4,0.8,21.8
3,44.0,1953-01-13 22:59:30,hospital_A,74575.24,1953-01-13 22:59:30,1953-01-13 22:59:30,NaN,NaN,NaN,NaN,...,1953-01-13 22:59:30,1953-01-13,Tuesday,1,1953,22,No,5.4,0.8,21.8
4,9592.0,1953-01-14 00:24:18,hospital_A,355051.48,1953-01-14 00:24:18,1953-01-14 00:24:18,NaN,NaN,NaN,NaN,...,1953-01-14 00:24:18,1953-01-14,Wednesday,1,1953,0,No,5.4,0.8,21.8


### Saving the dataframes as `.parquet`:

In [117]:
# Saving the three dataframes as a parquet file to your directory
df_for_three_hour_pred.to_parquet('path/to/your/file/data_for_three_hour_pred.parquet')
df_for_ten_hour_pred.to_parquet('path/to/your/file/data_for_ten_hour_pred.parquet')
df_for_twenty_four_hour_pred.to_parquet('path/to/your/file/data_for_twenty_four_hour_pred.parquet')

## END.

___
___